In [180]:
%load_ext autoreload
%autoreload
%cd /home/axel/master/ais-visualization/src

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/axel/master/ais-visualization/src


In [181]:
from main import *

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np
from pathlib import Path

In [182]:
# alt du må gjøre er å endre disse, så tror jeg det skal gå
num_runs = 1
num_folds = 10
num_boost_rounds = 10
file_pre = "ex_boost_white/logg_dat_SemanticKaggleW90.json"

In [183]:
def _parse_boosting_info(prop_values: dict, prop_type: str, enum_name: str):
    cols = {}
    for p in prop_values:
        if type(p) == str:
            if len(cols) == 0:
                continue
            else:
                for k, v in cols.items():
                    v.append(pd.NA)
        else:
            memberships: dict = p[enum_name]
            for k, v in memberships.items():
                cur_val = cols.get(k)
                if cur_val is None:
                    cols[k] = [v]
                else:
                    cols[k].append(v)

    df_dict = {}
    for k, v in cols.items():
        # v = np.add.reduceat(v, np.arange(0, len(v), 5))
        df_dict[f"{prop_type}_{k}"] = v
    print(df_dict)
    return (prop_type, df_dict)

In [184]:
def gen_cor_w_noreg_cols(df: pd.DataFrame, key: str):
    n_cor = df[f"{key}_cor"]
    n_wrong = df[f"{key}_wrong"]
    n_no_reg = df[f"{key}_no_reg"]

    df[f"{key}_precision"] = n_cor/(n_cor+n_wrong)
    df[f"{key}_accuracy"] = n_cor/(n_cor+n_wrong+n_no_reg)

In [185]:

with open(f"./../../ais/logg_dat/{file_pre}", "r") as f:
    js_obj = json.load(f)
[(v, i["property_type"]) for i in js_obj["meta_props"] for k, v in enumerate(i) ]

metas = js_obj["meta_props"]

for idx, v in enumerate(metas):
    print(idx, v["property_type"])

boost_accuracy = metas[0]
boost_accuracy_test = metas[1]

(p, d) = _parse_boosting_info(boost_accuracy_test["prop_values"], boost_accuracy_test["property_type"], "MappedInts")
(m, l) = _parse_boosting_info(boost_accuracy["prop_values"], boost_accuracy["property_type"], "MappedInts")
df_dict = []
l.update(d)
ddf = pd.DataFrame(l)

gen_cor_w_noreg_cols(ddf, "BoostAccuracyTest")
gen_cor_w_noreg_cols(ddf, "BoostAccuracy")


0 BoostAccuracy
1 BoostAccuracyTest
2 FoldAccuracy
3 Runtime
{'BoostAccuracyTest_cor': [2198, 2160, 2145, 2179, 2213, 2231, 2232, 2218, 2270, 2309], 'BoostAccuracyTest_no_reg': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'BoostAccuracyTest_wrong': [1049, 1087, 1102, 1068, 1034, 1016, 1015, 1029, 977, 938]}
{'BoostAccuracy_cor': [20160, 20488, 20680, 21144, 21647, 21932, 22319, 22511, 22933, 23057], 'BoostAccuracy_no_reg': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'BoostAccuracy_wrong': [9427, 9099, 8907, 8443, 7940, 7655, 7268, 7076, 6654, 6530]}


In [186]:
i = 4
step = 5
ddf[i::step]["BoostAccuracy_accuracy"]

4    0.731639
9    0.779295
Name: BoostAccuracy_accuracy, dtype: float64

In [187]:
def get_accs_std(ddf, num_folds: int,  num_boost_rounds: int):
    stds_train = []
    accs_train = []
    stds_test = []
    accs_test = []
    step = num_folds
    for i in range(0, num_boost_rounds):
        # print(df[i - 10:i:][])
        stds_train.append(ddf[i::step]["BoostAccuracy_accuracy"].std())
        accs_train.append(ddf[i::step]["BoostAccuracy_accuracy"].mean())
        m = ddf[i::step]["BoostAccuracyTest_accuracy"]
        print(m)
        stds_test.append(ddf[i::step]["BoostAccuracyTest_accuracy"].std())
        accs_test.append(ddf[i::step]["BoostAccuracyTest_accuracy"].mean())
    return pd.DataFrame.from_dict({"accs_train": accs_train, "stds_train": stds_train, "accs_test": accs_test, "stds_test": stds_test})

In [188]:
fig = go.Figure()
df_t = ddf[:10:]
markers = ["circle", "square", "diamond", "cross", "star", "triangle-up", "triangle-down", "triangle-left", "triangle-right"]

def plot_boosting_train_test_acc(ddf: pd.DataFrame, num_folds, num_boost_rounds):
    n = 0
    boosts = get_accs_std(ddf, 10, 10)
    fig.add_trace(go.Scatter(x=np.arange(0, len(ddf)), y=boosts["accs_test"],
                        mode='lines+markers',
                        name='BoostAccuracyTest',
                        error_y=dict(
                                     type='data',  # value of error bar given in data coordinates
                                     array=boosts["stds_test"],
                                     visible=True),
                        marker=dict(
                                    symbol=markers[n],
                                    size=8,
                                    #angleref="previous",
                                        )))
    
    n+=1
    fig.add_trace(go.Scatter(x=np.arange(0, len(ddf)), y=boosts["accs_train"],
                        mode='lines+markers',
                        name='BoostAccuracyTrain',
                        error_y=dict(
                                     type='data',  # value of error bar given in data coordinates
                                     array=boosts["stds_test"],
                                     visible=True),
                        marker=dict(
                                    symbol=markers[n],
                                    size=8,
                                    #angleref="previous",
                                        )))
    n += 1
    

    
    fig.update_layout(height = 500)
    fig.update_xaxes(range=[-0.5,9.5])
    fig.update_layout(legend_title_text=f" Mean accuracy <br> of {num_boost_rounds} boosting rounds")
    write_image(fig, f"{file_pre}", f"agg_boost.png")

    fig.show()
plot_boosting_train_test_acc(ddf, 10, 10)


0    0.676933
Name: BoostAccuracyTest_accuracy, dtype: float64
1    0.665229
Name: BoostAccuracyTest_accuracy, dtype: float64
2    0.66061
Name: BoostAccuracyTest_accuracy, dtype: float64
3    0.671081
Name: BoostAccuracyTest_accuracy, dtype: float64
4    0.681552
Name: BoostAccuracyTest_accuracy, dtype: float64
5    0.687096
Name: BoostAccuracyTest_accuracy, dtype: float64
6    0.687404
Name: BoostAccuracyTest_accuracy, dtype: float64
7    0.683092
Name: BoostAccuracyTest_accuracy, dtype: float64
8    0.699107
Name: BoostAccuracyTest_accuracy, dtype: float64
9    0.711118
Name: BoostAccuracyTest_accuracy, dtype: float64


,accs_train,stds_train,accs_test,stds_test
0,0.681380,NaN,0.676933,NaN
1,0.692466,NaN,0.665229,NaN
2,0.698956,NaN,0.660610,NaN
3,0.714638,NaN,0.671081,NaN
4,0.731639,NaN,0.681552,NaN
5,0.741272,NaN,0.687096,NaN
6,0.754352,NaN,0.687404,NaN
7,0.760841,NaN,0.683092,NaN
8,0.775104,NaN,0.699107,NaN
9,0.779295,NaN,0.711118,NaN
